In [13]:
import pandas as pd
from geopy.distance import geodesic
from tqdm import tqdm
import numpy as np

In [14]:
geodesic((37.1492, 127.0615),(37.149299999999999, 127.061599999999999)).m

14.21574505645554

# 데이터 합치기 + 전처리
1. 데이터 4개 합치기
2. 25미터 각 지점간 상호 거리 계산
3. 너무 가까운 지점 데이터 삭제
4. 50미터 데이터 만들기

In [15]:
df1 = pd.read_csv("data/0_8856.csv")
df1 = df1[[i for i in df1.columns if "chaos" not in i and 
                                    "width_nearby_count" not in i and
                                    "cross_road_nearby_count" not in i]]
df2 = pd.read_csv("data/0_8856_barrier.csv").iloc[:,2:]
df3 = pd.read_csv("data/0_8856_fix.csv").iloc[:,2:]
data25 = pd.concat([df1,df2,df3],axis=1)
# data25[["lon", "lat"]].to_csv("data/learn_point25_index.csv")

In [16]:
# index_df = pd.read_csv("data/learn_point25_index.csv",index_col = 0)
# dist_df = pd.DataFrame(0,index = list(range(len(index_df))),columns = list(range(len(index_df))))

# for col in tqdm(range(1, len(index_df))):
#     x = index_df.iloc[col]
#     dist_df.iloc[:col,col] = index_df[:col].apply(lambda y: geodesic((y[1],y[0]),(x[1], x[0])).m, axis = 1)
# dist_df[dist_df == 0] = dist_df.T
# np.save("data/learn_point25_pairdist.npy", dist_df.values.round(5))


In [17]:
pairdist = np.load("data/learn_point25_pairdist.npy")
pairdist[np.arange(len(pairdist)), np.arange(len(pairdist))] = float("inf")

na_index = data25[data25.isna().sum(axis=1) != 0].index

pairdist[na_index,:] = float("inf")
pairdist[:,na_index] = float("inf")

In [18]:
for i in na_index:
    target_tmp = np.argmin(pairdist[i])
    tmp = data25.loc[i].isna()
    data25.loc[i, tmp] = data25.loc[target_tmp, tmp]

In [19]:
tmp25 = data25[["lon","lat"]].round(10).apply(lambda x: (x[0],x[1]) ,axis = 1)
tmp50 = pd.read_csv("data/learn_point50.csv")[["lon","lat"]].round(10).apply(lambda x: (x[0],x[1]) ,axis = 1)

In [20]:
value = []
index = []
for row in tmp50.values:
    tmp = data25[tmp25 == tuple(row)]
    if len(tmp) != 0:
        value.append(tmp[:1].values)
        index.append(list(tmp[:1].index)[0])
    else:
        print(row)
data50 = pd.DataFrame(np.concatenate(value,axis=0),columns  = data25.columns, index = index)

In [21]:
data50 = data50.rename(columns = {'bump_count0' : "shortest_bump_dist",
                                  'shortest_cross_count0' : "shortest_cross_dist",
                                  'shortest_traffic_signal_count0' : "shortest_traffic_signal_dist", 
                                  'shortest_sidewalk_count0':'shortest_sidewalk_dist'})
data25 = data25.rename(columns = {'bump_count0' : "shortest_bump_dist",
                                  'shortest_cross_count0' : "shortest_cross_dist",
                                  'shortest_traffic_signal_count0' : "shortest_traffic_signal_dist", 
                                  'shortest_sidewalk_count0':'shortest_sidewalk_dist'})


In [23]:
data25 = data25[~(data25[["lon","lat"]] * 10000).astype(int).duplicated()]
data50 = data50[~(data50[["lon","lat"]] * 10000).astype(int).duplicated()]

In [24]:
data25 = data25[~data25[["lon","lat"]].round(4).duplicated()]
data50 = data50[~data50[["lon","lat"]].round(4).duplicated()]


In [27]:
data25.reset_index().to_csv("data/feature25.csv",index=False)
data50.reset_index().to_csv("data/feature50.csv",index=False)